# Dripping temperature anomaly

Create Figure 3, A1, A2, and B2. This requires temperature and meridional velocity for the simulations:

- TCo639_free_running/20060117_91L (hopg)
- TCo639_free_running/20060117_137L (hpp0)
- TCo639_free_running/20060117_198L (hopf)
- TCo639_free_running/20100205_91L (hopd)
- TCo639_free_running/20100205_137L (hpp1)
- TCo639_free_running/20100205_198L (hope)
- TCo639_free_running/20180208_91L (hokw)
- TCo639_free_running/20180208_137L (hpoz)
- TCo639_free_running/20180208_198L (hokx)

It also requires processed reanalysis data for climatologies or polar cap temperature anomalies and planetary wave meridional heat flux, as well as timeseries for the SSW events in 2006, 2010, and 2018.

In [ ]:
import numpy as np
import xarray as xr
import dask
import scipy.stats as stats
import scipy.ndimage
import numba
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import cmocean
import os

plt.rcParams.update({'font.size': 14})


In [ ]:
def load_Grib(directory,chunks={}):
    '''
    '''
    files = [directory+f for f in os.listdir(directory) if f.endswith('.grb')]
    files.sort()
    ds = xr.open_mfdataset(files,engine='cfgrib',chunks=chunks,combine='nested',concat_dim='number')
    
    return ds


def resample2daily(ds):
    '''
        Pandas' default behaviour for resampling is to put the time stamp at the beginning of the bin.
    '''
    # Rename dimensions
    try:
        ds = ds.drop(('step','time')).set_index(step='valid_time').rename(step='time')
    except:
        print('Exception: Time dimension is not renamed')
    if 'isobaricInhPa' in ds.dims:
        ds = ds.rename(isobaricInhPa='level')
        
    attrs = {}
    for var in ds.data_vars:
        try:
            attrs.update({var:dict(standard_name=ds[var].attrs['standard_name'],units=ds[var].attrs['units'])})
        except KeyError:
            try:
                attrs.update({var:dict(standard_name=ds[var].attrs['long_name'],units=ds[var].attrs['units'])})
            except:
                print('Exception: No name attribute') 
    
    # Resample to daily mean
    ds = ds.resample(time='1D').mean()

    for var in ds.data_vars:
        try:
            ds[var].attrs = attrs[var]
        except:
            # No name attribute
            print('')
        
    return ds


def area_weighted_mean(da,dim=('latitude','longitude')):
    '''
        Weight data on regular lon-lat grid with cosine of latitude
    '''
    weights = np.cos(da['latitude'] * np.pi/180)
    da = da * weights
    da = da.mean(dim=dim)
    da = da / weights.mean(dim='latitude')
    return da
    

def t_polar_anomal(sample,area = {'latitude':slice(90,60)}):
    '''
        Polar cap tempearture anomaly
    '''
    t = sample['t'].sel(**area)
    t = area_weighted_mean(t)
    t = t.compute()
    clim = xr.open_dataset(work_dir+'reanalysis_t_climatology.nc')
    clim = clim.sel(**area)
    clim = area_weighted_mean(clim)
    t = t.groupby('time.dayofyear') - clim['t']
    return t

In [ ]:
def wavenumber_decomposition(da,wavenum):
    '''
    '''
    lon = da.longitude
    k = xr.DataArray(np.fft.rfftfreq(len(lon),d=1/len(lon)),dims=('wavenumber'))
    # function that accepts dummy argument
    rfft = lambda da,k: dask.array.fft.rfft(da)
    irfft = lambda da,lon: dask.array.fft.irfft(da)
    
    transform = xr.apply_ufunc(rfft,
                               da,k,
                               input_core_dims=[['longitude'],['wavenumber']],
                               output_core_dims=[['wavenumber'],],
                               dask='allowed',
                               output_dtypes=[np.complex_]
                              )
    transform['wavenumber'] = k
    
    decomposed = []
    for k in wavenum:
        tmp = transform.where(transform.wavenumber==k,other=0)
        tmp = xr.apply_ufunc(irfft,
                             tmp,lon,
                             input_core_dims=[['wavenumber'],['longitude']],
                             output_core_dims=[['longitude'],],
                             dask='allowed',
                             output_dtypes=[np.float_]
                            )
        decomposed.append(tmp.assign_coords(wavenumber=k))
        
    decomposed = xr.concat(decomposed,dim='wavenumber')
    decomposed = decomposed.sum('wavenumber')
    
    return decomposed

In [ ]:
def one_sample_t_test(x0,sample,dim='number',broadcast_dims=('time','level')):
    '''
        Comparing sample mean against true value
        
        H0: \mu = x0
        
        Returns the probability that x1 - x0 larger than the observed value could have occurded by chance
    '''
    n = len(sample[dim])
    x1 = sample.mean(dim)
    s1 = sample.var(dim)
    
    t = (x1-x0) / np.sqrt(s1/n)
    dof = n-1
    
    p = xr.apply_ufunc(stats.t.sf,t,dof,
                       input_core_dims=[broadcast_dims,broadcast_dims],
                       output_core_dims=[broadcast_dims],
                       output_dtypes=t.dtype)
    return p


def two_sample_t_test(sample1,sample2,dim='number',broadcast_dims=('time','level')):
    '''
        Comparing two samples means, independent samples
        
        Be \sigma_1^2 \ne \sigma_2^2, but n_1=n_2=n
        H0: \mu_1 = \mu_2
        
        Returns the probability that x1 - x2 larger than the observed value could have occurded by chance
    '''
    n = len(sample1[dim])
    x1 = sample1.mean(dim)
    s1 = sample1.var(dim)
    x2 = sample2.mean(dim)
    s2 = sample2.var(dim)
    
    t = (x1-x2) / np.sqrt((s1+s2)/n)
    dof = n - 1 + (2 * n - 2) / (s1 / s2 + s2 / s1)

    p = xr.apply_ufunc(stats.t.sf,t,dof,
                       input_core_dims=[broadcast_dims,broadcast_dims],
                       output_core_dims=[broadcast_dims],
                       output_dtypes=t.dtype)
    return p


@numba.guvectorize(
    "(float64[:], float64[:], float64[:])",
    "(m), (n) -> (m)",
    forceobj=True
)
def vectorized_convolution(x,kernel,out):
    '''
        Vectorized convolution -> generalized NumPy universal function
        
        - mode='wrap' means that input is assumed being periodic
    '''
    out[:] = scipy.ndimage.convolve(x,kernel,mode='wrap')


def lowpass(da,dim,co,valid=False):
    '''
        convolution in time space is multiplication in frequency space
        
        - no chunking along core dimension dim
    '''
    # transform of Hanning window as better spectral properties than transfrom of box-car-window
    n = 2 * co  + 1
    hann = np.hanning(n)
    hann /= hann.sum()
    hann = xr.DataArray(hann,dims=('kernel'))
    
    filtered = xr.apply_ufunc(vectorized_convolution,
                              da,hann,                                           
                              input_core_dims=[[dim,],['kernel']],
                              output_core_dims=[[dim,],],
                              dask='parallelized',
                              output_dtypes=[da.dtype])
    
    # input is assumed to be periodic
    # remove beginning and end if unvalid
    if valid:
        valid_slice = slice((n - 1) // 2, -(n - 1) // 2)
        filtered = filtered.isel(time=valid_slice)
        
    return filtered


def climatology(da,ref):
    clim = da.groupby('time.dayofyear').mean('time').compute()
    clim = lowpass(clim,'dayofyear',30)
    clim = clim.where(clim['dayofyear'].isin(ref['time.dayofyear'].values),drop=True)
    clim = clim.drop('dayofyear').rename(dayofyear='time').assign_coords(time=ref.time)
    return clim

## Processing data

In [ ]:
# store polar cap temperature anomalies to zwischenspeicher
# model configurations

directory = lambda date, config: data_dir+date+'_'+config+'/pressure_levels_F64/t/'

startdates = ['TCo639_free_running/20180208',
              'TCo639_free_running/20060117',
              'TCo639_free_running/20100205'
              ]
configurations = ['91L','198L','137L',]

for date in startdates:
    for config in configurations:
        
        sample = load_Grib(directory(date,config))
        sample = resample2daily(sample)
        paint = t_polar_anomal(sample)
        
        filename = date+'_'+config+'_paint_t_polar_anomal.nc'
        print(filename)
        xr.Dataset(dict(t=paint)).to_netcdf(work_dir+filename)

In [ ]:
experiments = ['TCo639_free_running/20060117_91L',
               'TCo639_free_running/20060117_198L',
               'TCo639_free_running/20100205_198L',
               'TCo639_free_running/20100205_91L',
               'TCo639_free_running/20180208_91L',
               'TCo639_free_running/20180208_198L',
               ]

for exp in experiments:
    
    print(exp)
    
    v = load_Grib(data_dir+exp+'/pressure_levels_F64/v/',chunks=dict(step=16))
    v = resample2daily(v)['v']
    v = wavenumber_decomposition(v,[1,2,3]).persist()
    
    t = load_Grib(data_dir+exp+'/pressure_levels_F64/t/',chunks=dict(step=16))
    t = resample2daily(t)['t']
    t = wavenumber_decomposition(t,[1,2,3]).persist()
    flux = v * t
    flux = flux.mean('longitude').compute()
    
    xr.Dataset(dict(flux=flux)).to_netcdf(work_dir+exp+'_planetary_heat_flux.nc')

## Figure 3

Polar-cap temperature anomalies averaged between 60 and 90° N during the 2018 SSW event. The central date of the event is marked by the reversal of zonal-mean westerlies at 10 hPa and 60° N indicated by the vertical line. (a) The development of temperature anomalies [K] in the reanalysis, (b) the ensemble-mean bias of TCo639L91 hindcasts initialized on 8 February compared to the reanalysis, and (c) the improvement in the TCo639L198 hindcast
ensemble mean compared to the TCo639L91 hindcasts. Hatching indicates areas where ensemble-mean differences are not significantly different from zero at a 95% confidence level estimated by a one-sample (b) or two-sample t test (c). Panel (d) shows the ensemble-mean meridional eddy heat flux by zonal wavenumbers 1 to 3 at 100 hPa averaged between 45 and 70° N as a measure of the planetary wave flux in the lower stratosphere for climatology (dashed blue), the event in the reanalysis (orange), and the two model configurations (green and red).

In [ ]:
def dripping_temperature(reanalysis,L91,L198,
                         reanalysis_flux,L91_flux,L198_flux,
                         central_date):
    '''
    '''
    # load temperature data and compute plotables
    ref = xr.open_dataarray(reanalysis)
    ref = ref.sortby('level',ascending=False)
    ref['time'] = ref['time'] - np.timedelta64(9,'h')
    
    paint1 = xr.open_dataarray(L91)
    
    bias = paint1.mean('number') - ref.interp_like(paint1,method='nearest')
    p = one_sample_t_test(ref, paint1)
    bias_sig = np.add(p < 0.025, p > 0.975)
    
    paint2 = xr.open_dataarray(L198)
    
    improvement = paint2.mean('number') - paint1.mean('number')
    p = two_sample_t_test(paint1, paint2)
    improvement_sig = np.add(p < 0.025, p > 0.975)
    
    # load temperature flux
    L91 = xr.open_dataarray(L91_flux)
    L91 = area_weighted_mean(L91.sel(latitude=slice(70,45),level=100),dim='latitude').mean('number').drop('level')
    L198 = xr.open_dataarray(L198_flux)
    L198 = area_weighted_mean(L198.sel(latitude=slice(70,45),level=100),dim='latitude').mean('number').drop('level')
    reanalysis = xr.open_dataarray(reanalysis_flux)
    reanalysis = reanalysis.interp(time=ref.time)
    reanalysis = area_weighted_mean(reanalysis.sel(latitude=slice(70,45),level=100),dim='latitude').drop('level')    
    
    
    clim = xr.open_dataarray(data_dir+'reanalysis/reanalysis_planetary_heat_flux.nc')
    clim = area_weighted_mean(climatology(clim.sel(latitude=slice(70,45),level=100),ref),dim='latitude').drop('level')

    
    # Plotting
    
    fig, axes = plt.subplots(nrows=4,sharex='all',figsize=(6,9))
    
    C1 = ref.plot.pcolormesh(ax=axes[0],x='time',cmap=cmocean.cm.balance,extend='both',
                        levels=np.linspace(-25,25,33),add_colorbar=False)
    axes[0].plot([np.datetime64(central_date),np.datetime64(central_date)],[0,1000],'k-')
    
    ax = axes[0].twinx()
    ax.set_yticks([])
    ax.set_ylabel('reanalysis',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    xlim = axes[0].get_xlim()
    
    
    C2 = bias.plot.pcolormesh(ax=axes[1],x='time',cmap=cmocean.cm.balance,extend='both',
                        levels=np.linspace(-6,6,33),add_colorbar=False)
    bias_sig.astype(np.double).plot.contourf(ax=axes[1],x='time',levels=[0,0.5,1],hatches=['\\',''],
                                             alpha=0,add_colorbar=False)
    
    ax = axes[1].twinx()
    ax.set_yticks([])
    ax.set_ylabel('L91 bias',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    
    improvement.plot.pcolormesh(ax=axes[2],x='time',cmap=cmocean.cm.balance,extend='both',
                               levels=np.linspace(-6,6,33),add_colorbar=False)
    improvement_sig.astype(np.double).plot.contourf(ax=axes[2],x='time',levels=[0,0.5,1],hatches=['\\',''],
                                                   alpha=0,add_colorbar=False)
    
    ax = axes[2].twinx()
    ax.set_yticks([])
    ax.set_ylabel('L198 - L91',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    clim.plot.line(ax=axes[3],label='climatology',linestyle='dotted')
    reanalysis.plot.line(ax=axes[3],label='reanalysis',linestyle='solid')
    L91.plot.line(ax=axes[3],label='TCo639L91',linestyle='dashed')
    L198.plot.line(ax=axes[3],label='TCo639L198',linestyle='dashdot')
    axes[3].legend()
    
    ax = axes[3].twinx()
    ax.set_yticks([])
    ax.set_ylabel('heat flux',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    
    for ax in axes[:3]:
        ax.set_yscale('log')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax.set_ylabel('pressure [hPa]')
        ax.set_xlabel(None)
        ax.set_xlim(xlim)
    
    axes[3].set_ylabel(r'[K m s$^{-1}$]')
    axes[3].set_xlim(xlim)
    axes[3].set_xlabel('time')
    
    trans = mtransforms.ScaledTranslation(-45/72, -20/72, fig.dpi_scale_trans)
    
    axes[0].text(-0.06,1.0,'a)',transform=axes[0].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    axes[1].text(-0.06,1.0,'b)',transform=axes[1].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    axes[2].text(-0.06,1.0,'c)',transform=axes[2].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    axes[3].text(-0.06,1.0,'d)',transform=axes[3].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    
    fig.subplots_adjust(0,0,1,0.95,0,0)
    
    cbar = plt.colorbar(C1,ax=axes[0],orientation='vertical',fraction=0.1,aspect=10,shrink=0.95)
    cbar.set_label(r'[K]')
    
    cbar = plt.colorbar(C2,ax=axes[1:3],orientation='vertical',fraction=0.1,shrink=0.95)
    cbar.set_label(r'[K]')
    
    box = axes[3].get_position().get_points()
    box[1,0] = axes[2].get_position().get_points()[1,0]
    axes[3].set_position(box.flatten())
    
    
dripping_temperature(work_dir+'20180129_era5_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20180208_91L_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20180208_198L_paint_t_polar_anomal.nc',
                     work_dir+'era5_2018_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20180208_91L_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20180208_198L_planetary_heat_flux.nc',
                     '2018-02-12',)

## Figure A1

Same as Fig. 3 but for the SSW event in 2006.

In [ ]:
dripping_temperature(work_dir+'20060103_era5_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20060117_91L_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20060117_198L_paint_t_polar_anomal.nc',
                     work_dir+'era5_2006_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20060117_91L_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20060117_198L_planetary_heat_flux.nc',
                     '2006-01-21',)

## Figure A2

Same as Fig. 3 but for the SSW event in 2010.

In [ ]:
dripping_temperature(work_dir+'20100122_era5_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20100205_91L_paint_t_polar_anomal.nc',
                     work_dir+'TCo639_free_running/20100205_198L_paint_t_polar_anomal.nc',
                     work_dir+'era5_2010_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20100205_91L_planetary_heat_flux.nc',
                     work_dir+'TCo639_free_running/20100205_198L_planetary_heat_flux.nc',
                     '2010-02-09',)

## Figure B2

Differences in the TCo639L198 hindcast ensemblemean polar-cap temperature anomalies during the 2006, the 2010, and the 2018 SSW events compared to the TCo639L137 hindcasts.

In [ ]:
def plot_three_differences(L137_1,L198_1,
                           L137_2,L198_2,
                           L137_3,L198_3,):
    
    # load temperature data and compute plotables for first startdate
    paint1 = xr.open_dataarray(L137_1)
    paint2 = xr.open_dataarray(L198_1)
    
    difference_1 = paint2.mean('number') - paint1.mean('number')
    p = two_sample_t_test(paint1, paint2)
    sig_1 = np.add(p < 0.025, p > 0.975)
    
    # load temperature data and compute plotables for second startdate
    paint1 = xr.open_dataarray(L137_2)
    paint2 = xr.open_dataarray(L198_2)
    
    difference_2 = paint2.mean('number') - paint1.mean('number')
    p = two_sample_t_test(paint1, paint2)
    sig_2 = np.add(p < 0.025, p > 0.975)
    
    # load temperature data and compute plotables for third startdate
    paint1 = xr.open_dataarray(L137_3)
    paint2 = xr.open_dataarray(L198_3)
    
    difference_3 = paint2.mean('number') - paint1.mean('number')
    p = two_sample_t_test(paint1, paint2)
    sig_3 = np.add(p < 0.025, p > 0.975)
    
    # plotting
    fig, axes = plt.subplots(nrows=3,ncols=1,sharex=False,sharey=True,figsize=(8,12))
    axes = axes.flatten()
    
    C = difference_1.plot.pcolormesh(ax=axes[0],x='time',cmap=cmocean.cm.balance,extend='both',
                        levels=np.linspace(-6,6,33),add_colorbar=False)
    sig_1.astype(np.double).plot.contourf(ax=axes[0],x='time',levels=[0,0.5,1],hatches=['\\',''],
                                          alpha=0,add_colorbar=False)
    
    difference_2.plot.pcolormesh(ax=axes[1],x='time',cmap=cmocean.cm.balance,extend='both',
                        levels=np.linspace(-6,6,33),add_colorbar=False)
    sig_2.astype(np.double).plot.contourf(ax=axes[1],x='time',levels=[0,0.5,1],hatches=['\\',''],
                                          alpha=0,add_colorbar=False)
    
    difference_3.plot.pcolormesh(ax=axes[2],x='time',cmap=cmocean.cm.balance,extend='both',
                        levels=np.linspace(-6,6,33),add_colorbar=False)
    sig_3.astype(np.double).plot.contourf(ax=axes[2],x='time',levels=[0,0.5,1],hatches=['\\',''],
                                          alpha=0,add_colorbar=False)
    
    for ax in axes:
        ax.set_yscale('log')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax.set_ylabel('pressure [hPa]')
        ax.set_xlabel(None)
    axes[2].set_xlabel('time')
        
    trans = mtransforms.ScaledTranslation(-45/72, -20/72, fig.dpi_scale_trans)
    
    axes[0].text(-0.06,1.0,'a)',transform=axes[0].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    axes[1].text(-0.06,1.0,'b)',transform=axes[1].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    axes[2].text(-0.06,1.0,'c)',transform=axes[2].transAxes+trans,fontsize='large',va='bottom',fontfamily='serif')
    
    fig.subplots_adjust(0,0,1,1,0,0.4)
    
    ax = axes[0].twinx()
    ax.set_yticks([])
    ax.set_ylabel('2006',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    ax = axes[1].twinx()
    ax.set_yticks([])
    ax.set_ylabel('2010',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    ax = axes[2].twinx()
    ax.set_yticks([])
    ax.set_ylabel('2018',labelpad=80,size=18,weight='bold')
    ax.yaxis.set_label_position('left')
    
    cbar = plt.colorbar(C,ax=axes,orientation='horizontal',fraction=0.05,pad=0.1)
    cbar.set_label(r'L198 - L137 [K]')
    
    
plot_three_differences(work_dir+'TCo639_free_running/20060117_137L_paint_t_polar_anomal.nc',
                       work_dir+'TCo639_free_running/20060117_198L_paint_t_polar_anomal.nc',
                       work_dir+'TCo639_free_running/20100205_137L_paint_t_polar_anomal.nc',
                       work_dir+'TCo639_free_running/20100205_198L_paint_t_polar_anomal.nc',
                       work_dir+'TCo639_free_running/20180208_137L_paint_t_polar_anomal.nc',
                       work_dir+'TCo639_free_running/20180208_198L_paint_t_polar_anomal.nc',)